In [ ]:
#

In [ ]:
#

In [ ]:
import pandas as pd

import torch
import numpy as np
from numpy import pad
import pickle, os
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

import time
import json
import pandas as pd
from sklearn.metrics import roc_auc_score


In [ ]:

device='cpu'
sentence_transformer_models_dict = {}

In [ ]:
!pwd

In [ ]:
%run utils.ipynb

In [ ]:
%run config.ipynb

In [ ]:
EVAL_MODEL_LIST = ['all-mpnet-base-v2','all-distilroberta-v1','all-MiniLM-L6-v2', 'gemini','multi-qa-distilbert-cos-v1','paraphrase-MiniLM-L6-v2','distiluse-base-multilingual-cased-v1','distiluse-base-multilingual-cased-v2','paraphrase-multilingual-MiniLM-L12-v2','msmarco-distilbert-cos-v5','multi-qa-mpnet-base-cos-v1','text-embedding-3-small','text-embedding-3-large','voyage-3-lite'] #'meta-llama/Llama-2-7b-chat-h
# EVAL_MODEL_LIST =['all-mpnet-base-v2']# ['all-mpnet-base-v2','paraphrase-multilingual-MiniLM-L12-v2']
EVAL_MODEL_LIST+=['thenlper/gte-large','thenlper/gte-base','intfloat/e5-base-v2','embaas/sentence-transformers-e5-large-v2']#,'jinaai/jina-embeddings-v2-base-en']

# EVAL_MODEL_LIST = ['Mistral']

In [ ]:
BASEPATH = "STS_benchmark_paraphrase"

In [ ]:
REPLACEMENT_OR_REMOVAL = 'REMOVAL'

In [ ]:
#
LLM_FOR_ANONYMIZATION =['gemini']   # ['sonnet','']

In [ ]:
dataset_name = 'mrpc'

In [ ]:

df_all = pd.read_pickle(f'{BASEPATH}/pd_dataset_contains_person_name_{dataset_name}.pkl')
display(df_all)

In [ ]:
len(df_all)

In [ ]:

# "['REPLACEMENT', 'REMOVAL']:
for REPLACEMENT_OR_REMOVAL in  ['REMOVAL']:

    for llm_used in  LLM_FOR_ANONYMIZATION:
        if llm_used == 'gemini':
            delay=1
            num_workers = 1
        else:
            num_workers = 3
            delay =0


        with ThreadPoolExecutor(max_workers=num_workers) as executor:

            print('s1')
            df_all[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = list(executor.map(lambda row: text_anon({'text': row}, model=llm_used, delay=delay, REPLACEMENT_OR_REMOVAL=REPLACEMENT_OR_REMOVAL), df_all['sentence1']))


            print('s2')
            df_all[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = list(executor.map(lambda row: text_anon({'text': row},model=llm_used, delay=delay, REPLACEMENT_OR_REMOVAL=REPLACEMENT_OR_REMOVAL), df_all['sentence2']))


In [ ]:
display(df_all)

In [ ]:
df_all.to_pickle(f'{BASEPATH}/sts_benchmark_dataset_anon_{dataset_name}.pkl')

In [ ]:
df_all = pd.read_pickle(f"{BASEPATH}/sts_benchmark_dataset_anon_{dataset_name}.pkl")

In [ ]:
# ['REPLACEMENT', 'REMOVAL']:

for REPLACEMENT_OR_REMOVAL in ['REMOVAL']: #['REPLACEMENT']:

    for llm_used in LLM_FOR_ANONYMIZATION:

        df_all[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(get_text_after_modified_text)
        df_all[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(get_text_after_modified_text)


        df_all['sentence1_length'] = df_all['sentence1'].apply(lambda x: len(x.split()))
        df_all['sentence2_length'] = df_all['sentence2'].apply(lambda x: len(x.split()))

        df_all[f'anon_story1_length_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(lambda x: len(x.split()))
        df_all[f'anon_story2_length_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(lambda x: len(x.split()))



In [ ]:
display(df_all.head(10))

In [ ]:
EVAL_MODEL_LIST

In [ ]:
display(df_all)

In [ ]:
#

dict_model_wise = {}
for llm_used in LLM_FOR_ANONYMIZATION:

    correlation_df_llm_wise = pd.DataFrame(columns=['model', f'auc_roc_s1_s2',f'auc_roc_s1_s2_anonymized_llm_used_{llm_used}', f'SC_cosine_sim_score_s1_s2', f'SC_cosine_sim_score_s1_s2_anonymized_llm_used_{llm_used}','PC_cosine_sim_score_s1_s2', f'PC_cosine_sim_score_s1_s2_anonymized_llm_used_{llm_used}'])

    df_cop = df_all         #[df_all.sentence1_length >25]

    print('len ', len(df_cop))
    print('llm_used ', llm_used)

    REPLACEMENT_OR_REMOVAL = 'REMOVAL'

    for model_name in EVAL_MODEL_LIST:

        df_model = df_cop.copy()
        print('model_name ', model_name)
        print('machine summary')

        df_model[f'embedding_story1'] = np.array(embed_content(df_model['sentence1'].tolist(), model=model_name)).squeeze().tolist()
        df_model[f'embedding_story2'] = np.array(embed_content(df_model['sentence2'].tolist(), model=model_name)).squeeze().tolist()


        df_model[f'embedding_story1_anonymized_{model_name}_llm_used_{llm_used}'] = np.array(
            embed_content(df_model[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].tolist(), model=model_name)
        ).squeeze().tolist()

        df_model[f'embedding_story2_anonymized_{model_name}_llm_used_{llm_used}'] = np.array(
            embed_content(df_model[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].tolist(), model=model_name)
        ).squeeze().tolist()



        df_model['cosine_sim_story1_2'] = df_model.apply(lambda row: cosine_similarity(
        [np.array(row['embedding_story1'])], [np.array(row['embedding_story2'])])[0][0], axis=1)


        df_model[f'cosine_sim_story1_2_anon_llm_used_{llm_used}'] = df_model.apply(lambda row: cosine_similarity(
        [np.array(row[f'embedding_story1_anonymized_{model_name}_llm_used_{llm_used}'])], [np.array(row[f'embedding_story2_anonymized_{model_name}_llm_used_{llm_used}'])])[0][0], axis=1)



        # # Extract the required columns as arrays
        cosine_sim_score_anon_s1_s2 = df_model[f'cosine_sim_story1_2_anon_llm_used_{llm_used}'].explode().values
        cosine_sim_score_s1_s2 = df_model['cosine_sim_story1_2'].explode().values
        relevance_s1_s2 = df_model['label'].explode().values

        # # Compute Spearman and Pearson correlation between cosine_sim_score_anon and relevance
        spearman_corr_anon_relevance_s1_s2, _ = spearmanr(cosine_sim_score_anon_s1_s2, relevance_s1_s2)
        pearson_corr_anon_relevance_s1_s2, _ = pearsonr(cosine_sim_score_anon_s1_s2, relevance_s1_s2)

        # Compute Spearman and Pearson correlation between cosine_sim_score and relevance
        spearman_corr_cosine_relevance_s1_s2, _ = spearmanr(cosine_sim_score_s1_s2, relevance_s1_s2)
        pearson_corr_cosine_relevance_s1_s2, _ = pearsonr(cosine_sim_score_s1_s2, relevance_s1_s2)



        # # display(correlation_df)

        # print('llm used', llm_used)
        # print('->>>model ', model_name)
        print('::spearman_corr_cosine_relevance_s1_s2 ', spearman_corr_cosine_relevance_s1_s2)
        print('spearman_corr_anonymized_relevance_s1_s2 ', spearman_corr_anon_relevance_s1_s2)

        print('pearson_corr_cosine_relevance_s1_s2',pearson_corr_cosine_relevance_s1_s2 )
        print('pearson_corr_anonymized_relevance_s1_s2', pearson_corr_anon_relevance_s1_s2 )




        # Assuming df_model is a pandas DataFrame and 'label' is the true labels column
        # and 'cosine_sim_story1_2_anon_llm_used_cohere' is the predicted scores column
        y_true = df_model['label']
        y_scores_orig = df_model['cosine_sim_story1_2']

        y_scores_anon = df_model['cosine_sim_story1_2_anon_llm_used_cohere']

        auc_roc_anon = roc_auc_score(y_true, y_scores_anon)
        auc_roc_orig = roc_auc_score(y_true, y_scores_orig)
        print('auc_roc_orig', auc_roc_orig)
        print('auc_roc_anon ', auc_roc_anon)

        dict_model_wise[model_name] = df_model




In [ ]:
len(y_scores_anon), len(y_true)

In [ ]:
pd.set_option('display.max_colwidth', None)

dict_model_wise

In [ ]:
display(df_model[['label','cosine_sim_story1_2','cosine_sim_story1_2_anon_llm_used_cohere']].tail(40))